# Invoking the Deployed Model

In [14]:
%matplotlib inline
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing import image
from IPython.display import Image
import matplotlib.image as mpimg 
import matplotlib.pyplot as plt
import numpy as np
CIFAR10_LABELS = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

In [15]:
endpoint_name = 'tensorflow-cv-1656312038'
from sagemaker.tensorflow.model import TensorFlowPredictor
predictor = TensorFlowPredictor(endpoint_name = endpoint_name)

In [16]:
from tensorflow.keras.datasets import cifar10
(X_train, y_train), (X_test, y_test) = cifar10.load_data()
X_train = X_train.astype('float32')/255
X_test = X_test.astype('float32')/255

In [17]:
import threading
class WorkerThread(threading.Thread):
    def __init__(self, do_run, *args, **kwargs):
        super(WorkerThread, self).__init__(*args, **kwargs)
        self.__do_run = do_run
        self.__terminate_event = threading.Event()

    def terminate(self):
        self.__terminate_event.set()

    def run(self):
        while not self.__terminate_event.is_set():
            self.__do_run(self.__terminate_event)

In [39]:
import time
def invoke_endpoint(terminate_event):
    for i in range(10000):
        data = np.array([X_test[i]])
        payload = {'instances': data}
        response = predictor.predict(payload)
        time.sleep(1)
        if terminate_event.is_set():
            break

# Keep invoking the endpoint with test data
invoke_endpoint_thread = WorkerThread(do_run=invoke_endpoint)
invoke_endpoint_thread.start()

In [44]:
import random
from sagemaker.s3 import S3Downloader, S3Uploader
import pandas as pd
import uuid
MONITORING_FOLDER = 'DEMO-tf2-ModelMonitor'
BUCKET = 'sagemaker-us-east-1-949263681218'
data_capture_prefix = "{}/monitoring/ground_truth".format(MONITORING_FOLDER)
ground_truth_upload_path = "s3://{}/{}".format(BUCKET, data_capture_prefix)



def generate_load_and_ground_truth():
    df = pd.read_csv('validation_with_predictions.csv')
    gt_records = []
    for i, row in df.iterrows():
        suffix = uuid.uuid1().hex
        inference_id = f'{i}-{suffix}'
        out = predictor.predict(data = X_test[i],inference_id = inference_id)
        gt_records.append({
            "groundTruthData": {
                "data": str(df['label']),  # randomly generate positive labels 70% of the time
                "encoding": 'CSV',
            },
            "eventMetadata": {
                "eventId": str(inference_id),
            },
            "eventVersion": "0",
        })
    upload_ground_truth(gt_records, ground_truth_upload_path, datetime.utcnow())


def upload_ground_truth(records, path, upload_time):
    data_to_upload = ",".join(records)
    data_to_upload = '{' + data_to_upload + '}'
    target_s3_uri = f"{path}/{upload_time:%Y/%m/%d/%H/%M%S}.jsonl"
    print(f"Uploading {len(records)} records to", target_s3_uri)
    S3Uploader.upload_string_as_file_body(data_to_upload, target_s3_uri)


def generate_load_and_ground_truth_forever():
    while True:
        generate_load_and_ground_truth()
from threading import Thread
thread = Thread(target=generate_load_and_ground_truth_forever)
thread.start()

In [45]:
# Generate data for the last hour
from datetime import datetime, timedelta
import json
upload_ground_truth(datetime.utcnow() - timedelta(hours=1))

Uploading 10000 records to s3://sagemaker-us-east-1-949263681218/DEMO-tf2-ModelMonitor/monitoring/ground_truth/2022/06/27/03/4659.jsonl


In [46]:
# Generate data once a hour
def generate_fake_ground_truth(terminate_event):
    upload_ground_truth(datetime.utcnow())
    for _ in range(0, 60):
        time.sleep(60)
        if terminate_event.is_set():
            break


ground_truth_thread = WorkerThread(do_run=generate_fake_ground_truth)
ground_truth_thread.start()

Uploading 10000 records to s3://sagemaker-us-east-1-949263681218/DEMO-tf2-ModelMonitor/monitoring/ground_truth/2022/06/27/04/4700.jsonl


In [30]:
import boto3
sm = boto3.client('sagemaker')

In [13]:
!ls /opt/ml/

groundtruth  input_data  metadata


# JOB DESC

In [11]:
import boto3
import sagemaker
import os

region = boto3.Session().region_name
sm = boto3.Session().client("sagemaker")

tuning_job_name = 'tensorflow-training-220627-0556'

In [5]:
tuning_job_result = sm.describe_hyper_parameter_tuning_job(
    HyperParameterTuningJobName=tuning_job_name
)

status = tuning_job_result["HyperParameterTuningJobStatus"]
if status != "Completed":
    print("Reminder: the tuning job has not been completed.")

job_count = tuning_job_result["TrainingJobStatusCounters"]["Completed"]
print("%d training jobs have completed" % job_count)

objective = tuning_job_result["HyperParameterTuningJobConfig"]["HyperParameterTuningJobObjective"]
is_minimize = objective["Type"] != "Maximize"
objective_name = objective["MetricName"]

9 training jobs have completed


In [6]:
from pprint import pprint

if tuning_job_result.get("BestTrainingJob", None):
    print("Best model found so far:")
    pprint(tuning_job_result["BestTrainingJob"])
else:
    print("No training jobs have reported results yet.")

Best model found so far:
{'CreationTime': datetime.datetime(2022, 6, 27, 5, 56, 4, tzinfo=tzlocal()),
 'FinalHyperParameterTuningJobObjectiveMetric': {'MetricName': 'loss',
                                                 'Value': 1.0},
 'ObjectiveStatus': 'Succeeded',
 'TrainingEndTime': datetime.datetime(2022, 6, 27, 6, 0, 30, tzinfo=tzlocal()),
 'TrainingJobArn': 'arn:aws:sagemaker:us-east-1:949263681218:training-job/tensorflow-training-220627-0556-001-17c5ce0a',
 'TrainingJobName': 'tensorflow-training-220627-0556-001-17c5ce0a',
 'TrainingJobStatus': 'Completed',
 'TrainingStartTime': datetime.datetime(2022, 6, 27, 5, 57, 28, tzinfo=tzlocal()),
 'TunedHyperParameters': {'learning_rate': '0.10827078960526701'}}


In [30]:
!aws s3 cp --recursive s3://sagemaker-us-east-1-949263681218/DEMO-tf2-ModelMonitor/monitoring/ground_truth/ /opt/ml/groundtruth/.

download: s3://sagemaker-us-east-1-949263681218/DEMO-tf2-ModelMonitor/monitoring/ground_truth//2022/06/27/03/4537.jsonl to ../../opt/ml/groundtruth/2022/06/27/03/4537.jsonl
download: s3://sagemaker-us-east-1-949263681218/DEMO-tf2-ModelMonitor/monitoring/ground_truth//2022/06/27/03/3119.jsonl to ../../opt/ml/groundtruth/2022/06/27/03/3119.jsonl
download: s3://sagemaker-us-east-1-949263681218/DEMO-tf2-ModelMonitor/monitoring/ground_truth/2022/06/27/04/4700.jsonl to ../../opt/ml/groundtruth/2022/06/27/04/4700.jsonl
download: s3://sagemaker-us-east-1-949263681218/DEMO-tf2-ModelMonitor/monitoring/ground_truth//2022/06/27/04/3129.jsonl to ../../opt/ml/groundtruth/2022/06/27/04/3129.jsonl
download: s3://sagemaker-us-east-1-949263681218/DEMO-tf2-ModelMonitor/monitoring/ground_truth/2022/06/27/03/4659.jsonl to ../../opt/ml/groundtruth/2022/06/27/03/4659.jsonl
download: s3://sagemaker-us-east-1-949263681218/DEMO-tf2-ModelMonitor/monitoring/ground_truth//2022/06/27/04/4538.jsonl to ../../opt/ml/g

In [32]:
!ls /opt/ml/

groundtruth  input_data  metadata


In [12]:
region

'us-east-1'

In [21]:
!aws s3 ls sagemaker-us-east-1-949263681218/DEMO-tf2-ModelMonitor/monitoring/ground_truth/2022

                           PRE 06/


In [38]:
!ls /opt/ml/input_data
!ls /opt/ml/groundtruth

AllTraffic		  tensorflow-cv-1656042735  tensorflow-cv-1656302831
tensorflow-cv-1655979252  tensorflow-cv-1656064111  tensorflow-cv-1656312038
2022


In [ ]:
!aws s3 cp --recursive s3://sagemaker-us-east-1-949263681218/DEMO-tf2-ModelMonitor/monitoring/datacapture/ /opt/ml/input_data/